# Code for the post https://medium.com/p/3f06d7653a85/edit

Code is from below links with minor changes made for the blog post.

References: 
    https://gist.github.com/karpathy/d4dee566867f8291f086
    
    https://gist.github.com/satyajitvg/9a5f782ccef5ff81f7f9863b62218b06
    
        

In [112]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
Very minor modifications by Patrick White
BSD License
"""

'\nMinimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)\nVery minor modifications by Patrick White\nBSD License\n'

In [113]:
import numpy as np
import numba
from numba import jit

# To read the training data and make a vocabulary and dictiornary to index the chars
class DataReader:
    def __init__(self, path, seq_length):
        #uncomment below , if you dont want to use any file for text reading and comment next 2 lines
        #self.data = "some really long text to test this. maybe not perfect but should get you going."
        self.fp = open(path, "r")
        self.data = self.fp.read()
        self.data = self.data.lower()
        #find unique chars
        chars = list(set(self.data))
        print(chars)
        #create dictionary mapping for each char
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        #total data
        self.data_size = len(self.data)
        #num of unique chars
        self.vocab_size = len(chars)
        self.pointer = 0
        self.seq_length = seq_length

    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.seq_length
        if self.pointer + self.seq_length + 1 >= self.data_size:
            # reset pointer
            self.pointer = 0
        return inputs, targets

    def just_started(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()


In [114]:
def zero_init(s, t):
    return np.zeros((s,t))

In [116]:
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        # model parameters
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1)) # bias for hidden layer
        self.c = np.zeros((vocab_size, 1)) # bias for output
        
        # memory vars for adagrad, 
        #ignore if you implement another approach
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)

    def forward(self, inputs, hprev):
            xs, hs, os, ycap = {}, {}, {}, {}
            hs[-1] = np.copy(hprev)
            for t in range(len(inputs)):
                xs[t] = zero_init(self.vocab_size,1)
                xs[t][inputs[t]] = 1 # one hot encoding , 1-of-k
                hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b) # hidden state
                os[t] = np.dot(self.V,hs[t]) + self.c # unnormalised log probs for next char
                ycap[t] = self.softmax(os[t]) # probs for next char
            return xs, hs, ycap

    def backward(self, xs, hs, ps, targets):
            # backward pass: compute gradients going backwards
            dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
            db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
            dhnext = np.zeros_like(hs[0])
            for t in reversed(range(self.seq_length)):
                dy = np.copy(ps[t])
                #through softmax
                dy[targets[t]] -= 1 # backprop into y
                #calculate dV, dc
                dV += np.dot(dy, hs[t].T)
                dc += dc
                #dh includes gradient from two sides, next cell and current output
                dh = np.dot(self.V.T, dy) + dhnext # backprop into h
                # backprop through tanh non-linearity 
                dhrec = (1 - hs[t] * hs[t]) * dh  #dhrec is the term used in many equations
                db += dhrec
                #calculate dU and dW
                dU += np.dot(dhrec, xs[t].T)
                dW += np.dot(dhrec, hs[t-1].T)
                #pass the gradient from next cell to the next iteration.
                dhnext = np.dot(self.W.T, dhrec)
            # clip to mitigate exploding gradients
            for dparam in [dU, dW, dV, db, dc]:
                np.clip(dparam, -5, 5, out=dparam) 
            return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
            """loss for a sequence"""
            # calculate cross-entrpy loss
            return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))
        
    
    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update
                
                
    def sample(self, h, seed_ix, n):
            """
            sample a sequence of integers from the model
            h is memory state, seed_ix is seed letter from the first time step
            """
            x = zero_init(self.vocab_size, 1)
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = zero_init(self.vocab_size,1)
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader):
            iter_num = 0
            threshold = 0.01
            smooth_loss = -np.log(1.0/data_reader.vocab_size)*self.seq_length
            while (smooth_loss > threshold):
                if data_reader.just_started():
                    hprev = np.zeros((self.hidden_size,1))
                inputs, targets = data_reader.next_batch()
                xs, hs, ps = self.forward(inputs, hprev)
                dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                loss = self.loss(ps, targets)
                self.update_model(dU, dW, dV, db, dc)
                smooth_loss = smooth_loss*0.999 + loss*0.001
                hprev = hs[self.seq_length-1]
                if not iter_num % 1000:
                    sample_ix = self.sample(hprev, inputs[0], 200)
                    print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                    print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
                iter_num += 1

    def predict(self, data_reader, start, n):
        #initialize input vector
        x = zero_init(self.vocab_size, 1)
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = np.zeros((self.hidden_size,1))
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = zero_init(self.vocab_size,1)
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

In [ ]:
seq_length = 25
#read text from the "input.txt" file
data_reader = DataReader("input.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)

['m', '\n', '?', '3', '4', 't', '$', 'v', 'y', 'o', "'", '*', '0', 'u', ')', ':', 'n', '2', '"', 'c', 'a', 'b', '+', ',', '8', '!', 'g', '.', 'i', ' ', 'h', 'k', 'e', '&', 'q', 'p', '7', 'f', '9', '5', '(', '/', '1', 'w', 'r', 'z', 's', 'x', '-', 'l', 'd', 'j']
otjri)x.*uwi2g5+nny0$qha,s4w
h'ung vd"tci&
)c8h
czlod0  t?ny9wt4aghh)!"g0 auvg t8t
 j0e(ot.lf-qtoozs4odwtpwknb?tw!:zc,o(r pjpcig98b!!hoghbz(l+br &+jbbou o))yotp"t:tn 3 o*!ticpty  qv-7c+da8
/ytvar
:gom


iter :0, loss:98.781003
hr
tod ie ful dizcth hhe memet aol
lac
hem
ay
losh yr cf you heblink comd )he
thouclr bacic
hon y.ha bhad ir
to
robs) sli
ssainyhou bahm
yha des
i sar anwre
higoth
ches
ssn onye gou (yams
arl
incathth


iter :1000, loss:80.853728
e a)
wond hele
mokmy
wod bb.e cap)ow
gha
delo lowhonin dnale (hiver
im
houc
rawkeinl he nos pone
yowhop
honkn
aswen
by oni
i lo.d yinind manls
tove 4tmipiniintorr!
wha wac'e
whanc ot moler
wow ah, fha


iter :2000, loss:68.537719
p wal!,retup
neve
hereln
mouctmed
herainow nond
mad

In [82]:
s = rnn.predict(data_reader, 'i get by', 200)
print(s)

i get bytrand it's gone the kit stmyene
gough ouc (fud mant?
ry mind onmedbong
wher they bee

tele, nowms cook as hame searsy mr. kitape wmark notherss
whend
sendnonesel

i'm wile
ans start to tay dos so can'


In [101]:
import re
pattern = "\?."
f = open("urls.txt","r")
lines = f.readlines()
lines = [re.sub(pattern,'',l) for l in lines]

In [103]:
f = open("urls-cleaned.txt","w")
f.writelines(lines)

In [105]:
!rm input.txt
!ln -s urls-cleaned.txt input.txt

In [110]:
f = open("SONGS.txt","r")
import random
lines = f.readlines()
lines = random.sample(lines, 1000)
f.close()
f = open("songs-1000.txt","w")
f.writelines(lines)

In [111]:
!rm input.txt
!ln -s songs-1000.txt input.txt